# temperature 가져오기

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate
from utils import get_openai_api_key
from Tools.temperature import get_current_temperature
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish
import openai

openai.api_key = get_openai_api_key()
functions = [
        format_tool_to_openai_function(f) for f in [get_current_temperature]
]
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant. If user ask you about movie, you should answer sincerely. \
        if you don't have information of user, suppose that the user is in Seoul and don't ask user"),
    ("user", "{input}"),
])

def route(result):
	if isinstance(result, AgentFinish): # 함수를 쓰지 않기로 결정한다면 -> content 반환
		return result.return_values['output']
	else: # 함수를 쓰기로 결정한다면 -> 함수명에 따라 어떤 함수를 쓸지 결정해주고, argument를 넣은 값 반환
		tools = {
			"get_current_temperature": get_current_temperature,
		}
		return tools[result.tool].run(result.tool_input)
  
model = ChatOpenAI(temperature=0).bind(functions=functions)
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route
chain.invoke({"input":"오늘 부산 날씨는 어떤것 같아"})

'The current temperature is -3.5°C'

In [3]:
prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a movie recommender. Use your knowledge to recommend movies that fit\
    and rewrite with given informations {informations}. \
    If needed, you can use latest movie informations {latest_movies}    \
    You shoud answer using only Korean and markdown structure"),
])

In [4]:
model2 = ChatOpenAI(temperature=0, max_tokens=1024)
chain2 = prompt2 | model2

In [7]:
result1 = informations=chain.invoke({"input":"오늘 날씨에 맞는 영화 추천좀"})
result2 = chain2.invoke({"informations": result1})

In [9]:
print(result2.content)

현재 온도는 -8.2°C입니다. 추운 날씨에는 온기를 느낄 수 있는 영화를 추천해드릴게요. 아래의 영화들을 즐기시면 추위를 잊고 영화 속에서 따뜻한 분위기를 느낄 수 있을 거예요:

1. 겨울왕국 (Frozen): 겨울 풍경과 따뜻한 가족 사랑을 다룬 애니메이션 영화입니다. 눈과 얼음으로 뒤덮인 왕국에서 펼쳐지는 이야기는 추운 날씨에도 마음을 녹일 수 있어요.

2. 러브 액츄얼리 (Love Actually): 크리스마스 분위기와 사랑 이야기가 어우러진 영화입니다. 다양한 인물들의 이야기가 따뜻한 감동을 전해줄 거예요.

3. 캐롤 (Carol): 1950년대 겨울을 배경으로 한 로맨스 드라마입니다. 눈이 내리는 도시에서 펼쳐지는 사랑 이야기는 추운 날씨에 어울리는 영화 중 하나에요.

4. 홈 애로인 (Home Alone): 겨울 방학 동안 집에 혼자 남겨진 소년의 모험을 그린 코미디 영화입니다. 따뜻한 가족의 사랑과 유쾌한 이야기로 추위를 잊게 해줄 거예요.

이 영화들을 시청하면 추운 날씨에도 따뜻한 감정을 느낄 수 있을 거예요. 즐겁게 영화를 감상하세요!


# 다음 영화 크롤링

In [181]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import random
import time

# Selenium 웹 드라이버 설정 
driver = webdriver.Chrome()
options = Options()

#지정한 user-agent로 설정합니다.
user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
options.add_argument('user-agent=' + user_agent)
options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.

# 웹 페이지 열기
url = 'https://movie.daum.net/premovie/theater'
driver.get(url)

# 페이지가 완전히 로딩되도록 3초동안 기다림
time.sleep(3)

# 해당 경로의 요소 가져오기
ol_element = driver.find_element(By.XPATH, '//*[@id="mainContent"]/div/div[2]/ol')
# ol 아래의 모든 li 요소 가져오기
li_elements = ol_element.find_elements(By.TAG_NAME, 'li')
infos = []

# 가져온 li 요소 출력
for li in li_elements:
    li.click()
    # 0.4에서 0.6 사이의 랜덤한 값을 선택하여 sleep에 적용
    random_sleep_time = random.uniform(0.4, 0.6)
    time.sleep(random_sleep_time)
    infos.append(li.text)

# 웹 드라이버 종료
driver.quit()

In [182]:
infos

['12세이상관람가\n임진왜란 발발로부터 7년이 지난 1598년 12월. 이순신(김윤석)은 왜군의 수장이던 도요토미 히데요시가 갑작스럽게 사망한 뒤 왜군들이 조선에서 황급히 퇴각하려 한다는 것을 알게 된다. “절대 이렇게 전쟁을 끝내서는 안 된다” 왜군을 완벽하게 섬멸하는 것이 이 전쟁을 올바르게 끝나는 것이라 생각한 이순신은 명나라와 조명연합함대를 꾸려 왜군의 퇴각로를 막고 적들을 섬멸하기로 결심한다.하지만 왜군의 뇌물 공세에 넘어간 명나라 도독 진린(정재영)은 왜군에게 퇴로를 열어주려 하고,설상가상으로 왜군 수장인 시마즈(백윤식)의 살마군까지 왜군의 퇴각을 돕기 위해 노량으로 향하는데…2023년 12월, 모두를 압도할 최후의 전투가 시작된다!\n노량: 죽음의 바다\n평점8.4 예매율43.1%\n개봉23.12.20',
 '12세이상관람가\n1979년 12월 12일, 수도 서울 군사반란 발생그날, 대한민국의 운명이 바뀌었다대한민국을 뒤흔든 10월 26일 이후, 서울에 새로운 바람이 불어온 것도 잠시12월 12일, 보안사령관 전두광이 반란을 일으키고군 내 사조직을 총동원하여 최전선의 전방부대까지 서울로 불러들인다.권력에 눈이 먼 전두광의 반란군과 이에 맞선 수도경비사령관 이태신을 비롯한진압군 사이, 일촉즉발의 9시간이 흘러가는데…목숨을 건 두 세력의 팽팽한 대립오늘 밤, 대한민국 수도에서 가장 치열한 전쟁이 펼쳐진다!\n서울의 봄\n평점9.5 예매율25.9%\n개봉23.11.22',
 '12세이상관람가\n아틀란티스 왕국을 이끌 왕의 자리에 오른 ‘아쿠아맨’. 그 앞에 ‘블랙 만타’가 세상을 뒤흔들 강력한 지배 아이템 ‘블랙 트라이던트’를 손에 넣게 된다.그동안 겪지 못 했던 최악의 위협 속 ‘아쿠아맨’은‘블랙 만타’와 손을 잡았던 이부 동생 ‘옴’ 없이는 절대적 힘이 부족한 상황.바다를 지배할 슈퍼 히어로가 세상의 판도를 바꾼다! \n아쿠아맨과 로스트 킹덤\n평점6.3 예매율9.4%\n개봉23.12.20',
 '전체관람가\n최초의 3D CG! 제작 기간 7년 최

In [195]:
import re
from datetime import datetime
import pandas as pd

def parse_movie_info(movie_info_list):
    parsed_info = []
    for info in movie_info_list:
        split_info = info.split('\n')
        if len(split_info) == 5:
            if len(split_info[3].split())!=2:
                continue
            age_limit = split_info[0]
            plot = split_info[1]
            title = split_info[2]
            rating, booking_rate = (split_info[3].split())
            # rating = re.sub(r'\D', '',rating)
            # booking_rate = re.sub(r'\D', '',booking_rate)            
            release_date = re.sub(r'\D', '', split_info[4])
            release_date_formatted = datetime.strptime('20'+release_date, '%Y%m%d').strftime('%y년 %m월 %d일')
            
            parsed_info.append({
                '관람가': age_limit,
                '제목' : title,
                '평점': rating,
                '예매율': booking_rate,
                '개봉일': release_date_formatted,
                '줄거리': plot,
            })
    
    return pd.DataFrame(parsed_info)
parse_movie_info(infos)

,관람가,제목,평점,예매율,개봉일,줄거리
0,12세이상관람가,노량: 죽음의 바다,평점8.4,예매율43.1%,23년 12월 20일,임진왜란 발발로부터 7년이 지난 1598년 12월. 이순신(김윤석)은 왜군의 수장이...
1,12세이상관람가,서울의 봄,평점9.5,예매율25.9%,23년 11월 22일,"1979년 12월 12일, 수도 서울 군사반란 발생그날, 대한민국의 운명이 바뀌었다..."
2,12세이상관람가,아쿠아맨과 로스트 킹덤,평점6.3,예매율9.4%,23년 12월 20일,아틀란티스 왕국을 이끌 왕의 자리에 오른 ‘아쿠아맨’. 그 앞에 ‘블랙 만타’가 세...
3,전체관람가,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,평점10.0,예매율6.7%,23년 12월 22일,최초의 3D CG! 제작 기간 7년 최고의 웃음과 감동! 최강의 스케일 옷까지 갈아...
4,전체관람가,트롤: 밴드 투게더,평점10.0,예매율5.9%,23년 12월 20일,최애 아이돌 그룹 재결합 초읽기! 우리 함께하면 더 좋잖아! 전 세계 모든 트롤을 ...
5,전체관람가,뽀로로 극장판 슈퍼스타 대모험,평점10.0,예매율2.5%,23년 12월 13일,뽀로로와 친구들은 전 우주를 통틀어 최고의 슈퍼스타를 뽑는 음악 축제 ‘파랑돌 슈퍼...
6,12세이상관람가,괴물,평점8.0,예매율1.0%,23년 11월 29일,“우리 동네에는 괴물이 산다” 싱글맘 사오리(안도 사쿠라)는 아들 미나토(쿠로카와 ...
7,전체관람가,크레센도,평점10.0,예매율0.6%,23년 12월 20일,"“예술성, 테크닉, 기교, 드라마, 짜릿함. 그는 피아노 연주의 극치를 보여주었다!..."
8,12세이상관람가,3일의 휴가,평점7.8,예매율0.5%,23년 12월 06일,“따님은 어머님을 보거나 목소리를 들을 수 없고요. 그냥 따님의 행복한 기억만 담고...
9,전체관람가,말하고 싶은 비밀,평점9.4,예매율0.3%,23년 12월 13일,어느 날 갑자기 고백 사고!? 마음이 잘못 배달되었다! ‘노조미’는 자신의 책상 서...


In [202]:
import pandas as pd
prompt3 = ChatPromptTemplate.from_messages([
    ("system", "You are a movie recommender. Use your knowledge to recommend movies that fit\
    and rewrite with given informations {informations}. \
    If needed, you can use latest movie informations {latest_movies}    \
    You shoud answer using only Korean and markdown structure"),
])
model3 = ChatOpenAI(temperature=0, max_tokens=1024)
chain3 = prompt3 | model3
result1 = informations=chain.invoke({"input":"오늘 날씨에 맞는 영화 추천좀. 최신 영화중에 어떤게 좋을까?"})
result3 = chain3.invoke({"informations": result1, "latest_movies":pd.read_csv("./data/daum_movie/movie_list.csv")})

In [203]:
print(result3.content)

현재 온도가 0.4°C로 매우 추운 날씨입니다. 추위를 덜 느끼고 영화를 즐길 수 있는 영화를 추천해 드리겠습니다.

1. 노량: 죽음의 바다 - 평점 8.4, 예매율 42.8%
   - 임진왜란 시기의 이야기로, 전투와 모험을 담은 역사적인 액션 영화입니다.

2. 서울의 봄 - 평점 9.5, 예매율 25.9%
   - 1979년 군사반란이 일어난 서울의 운명을 바꾼 이야기로, 감동적인 드라마입니다.

3. 괴물 - 평점 8.0, 예매율 1.0%
   - 동네에 사는 싱글맘과 아들의 이야기로, 스릴과 감동을 함께 느낄 수 있는 영화입니다.

4. 3일의 휴가 - 평점 7.8, 예매율 0.4%
   - 청춘들의 여행 이야기로, 휴가를 가지고 싶은 마음을 담은 로맨틱 코미디입니다.

5. 싱글 인 서울 - 평점 8.2, 예매율 0.2%
   - 혼자 살아가는 도시인들의 이야기로, 서울에서의 싱글 라이프를 그린 로맨틱 드라마입니다.

이 추운 날씨에는 따뜻한 영화를 즐기며 따뜻한 감정을 느껴보세요!


In [205]:
model3 = ChatOpenAI(temperature=0.7, max_tokens=1024)
chain3 = prompt3 | model3
result1 = informations=chain.invoke({"input":"볼 만한 영화 없나?"})
result3 = chain3.invoke({"informations": result1, "latest_movies":pd.read_csv("./data/daum_movie/movie_list.csv")})
print(result3.content)

저는 12세 이상 관람가인 영화 '노량: 죽음의 바다'를 추천해드릴 수 있습니다. 이 영화는 임진왜란 시기의 이순신 장군을 다루고 있으며, 평점은 8.4이고 예매율은 42.8%입니다. 개봉일은 23년 12월 20일이며, 줄거리는 임진왜란 발발로부터 7년이 지난 1598년 12월, 이순신(김윤석)이 왜군의 수장이 된 이후의 이야기를 다룹니다. 다른 장르의 영화를 원하시면 알려주세요.


In [208]:
import pandas as pd
prompt3 = ChatPromptTemplate.from_messages([
    ("system", "You are a movie recommender. Use your knowledge to recommend movies that fit\
    and rewrite with given informations {informations}. \
    If needed, you can use latest movie informations {latest_movies} \
    You must recommned more than one movie.\
    You shoud answer using only Korean and markdown structure"),
])
model3 = ChatOpenAI(temperature=0, max_tokens=1024)
chain3 = prompt3 | model3
result1 = informations=chain.invoke({"input":"색다른 영화 없을까?"})
result3 = chain3.invoke({"informations": result1, "latest_movies":pd.read_csv("./data/daum_movie/movie_list.csv")})
print(result3.content)

액션, 로맨스, 코미디, 공포 등 어떤 장르를 원하시나요?


In [209]:
result1

'어떤 종류의 영화를 찾고 계세요? 액션, 로맨스, 코미디, 공포 등등 어떤 장르를 원하시나요?'

구체적이지 않은 쿼리에는 잘 답하지 못한다.

# 네이버 Search api 테스트

In [31]:
import requests
from utils import get_naver_api_key
client_id, client_secret = get_naver_api_key()
query = "노량"
encText = requests.utils.quote(query)
url = f"https://openapi.naver.com/v1/search/news"  # GET 요청 URL

headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret,
}

params = {
    'query' : encText,
    'display': 3,    # 표시할 검색 결과 수 (최대 100까지 가능)
    'start': 1,       # 검색 시작 위치
    'sort': 'sim',    # 정렬 옵션 ('sim': 유사도순, 'date': 날짜순)
}

response = requests.get(url, headers=headers, params=params)

if response.status_code == 200:
    result = response.json()
    print(result)
else:
    print("Error Code:", response.status_code)

Error Code: 400


In [29]:
# JSON 데이터를 예쁘게 출력
import json
pretty_json = json.dumps(result.get('items'), indent=4, ensure_ascii=False)
print(pretty_json)

[
    {
        "title": "제니가 공개한 깜짝 크리스마스 선물!",
        "originallink": "https://www.allurekorea.com/2023/12/24/%ec%a0%9c%eb%8b%88%ea%b0%80-%ea%b3%b5%ea%b0%9c%ed%95%9c-%ea%b9%9c%ec%a7%9d-%ed%81%ac%eb%a6%ac%ec%8a%a4%eb%a7%88%ec%8a%a4-%ec%84%a0%eb%ac%bc/?utm_source=naver&utm_medium=partnership",
        "link": "https://www.allurekorea.com/2023/12/24/%ec%a0%9c%eb%8b%88%ea%b0%80-%ea%b3%b5%ea%b0%9c%ed%95%9c-%ea%b9%9c%ec%a7%9d-%ed%81%ac%eb%a6%ac%ec%8a%a4%eb%a7%88%ec%8a%a4-%ec%84%a0%eb%ac%bc/?utm_source=naver&utm_medium=partnership",
        "description": "https://www.allurekorea.com/2023/10/06/%ec%a0%9c<b>%eb%</b>8b%88-<b>%eb%</b><b>89</b>%b4%ec%a7%84%ec%8a%a4-%ed%94%bd-%ed%8c%a<b>8%</b>ec%<b>85%</b>98-<b>%eb%</b><b>b8%</b>8c... b9%98%ec%9d%98-%ed%91%9c<b>%eb%</b>b3%b8-%ec%a0%9c<b>%eb%</b>8b%88%ec%9d%98-<b>%eb%</b><b>9f%</b>ad%ec%<b>85%</b>94<b>%eb%</b>a6%ac-%ec%9d%<b>b8%</b>ed%<b>85%</b>8c<b>%eb%</b>a6%ac%ec%96%b4",
        "pubDate": "Sun, 24 Dec 2023 18:20:00 +0900"
    },
    {
      

?? 검색 결과가 다름... 다른 Search api 고려..

# Serper API

In [64]:
from utils import get_serp_api_key
from serpapi import GoogleSearch

params = {
  "api_key": "6a4a923909dd8cf66cc47598a891f8ae1c1c88417f15062ded267489f0ac64b3",
  "engine": "google",
  "q": "크리스토퍼 놀란 최신영화",
  "location": "Seoul, Seoul, South Korea",
  "google_domain": "google.co.kr",
  "gl": "kr",
  "hl": "ko"
}

search = GoogleSearch(params)
results = search.get_dict()

In [65]:
results

{'search_metadata': {'id': '65892176d3aba296ddee4e13',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/165cad796156bf19/65892176d3aba296ddee4e13.json',
  'created_at': '2023-12-25 06:30:14 UTC',
  'processed_at': '2023-12-25 06:30:14 UTC',
  'google_url': 'https://www.google.co.kr/search?q=%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%86%A0%ED%8D%BC+%EB%86%80%EB%9E%80+%EC%B5%9C%EC%8B%A0%EC%98%81%ED%99%94&oq=%ED%81%AC%EB%A6%AC%EC%8A%A4%ED%86%A0%ED%8D%BC+%EB%86%80%EB%9E%80+%EC%B5%9C%EC%8B%A0%EC%98%81%ED%99%94&uule=w+CAIQICIXU2VvdWwsU2VvdWwsU291dGggS29yZWE&hl=ko&gl=kr&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/165cad796156bf19/65892176d3aba296ddee4e13.html',
  'total_time_taken': 1.37},
 'search_parameters': {'engine': 'google',
  'q': '크리스토퍼 놀란 최신영화',
  'location_requested': 'Seoul, Seoul, South Korea',
  'location_used': 'Seoul,Seoul,South Korea',
  'google_domain': 'google.co.kr',
  'hl': 'ko',
  'gl': 'kr',
  'device': 'desktop'},
 'search_inf

In [7]:
results['knowledge_graph']

{'title': '노량: 죽음의 바다',
 'type': '12세 이상 관람가 2023년 ‧ 액션/전쟁 ‧ 2시간 32분',
 'image': 'https://serpapi.com/searches/6588fccd5119a6f342fc4d6b/images/e4b4dc9825ab3874847816d2e1020ec5b4ae9eb068e96c13.jpeg',
 'description': '1598년 노량해협, 명량 해전 이후 도요토미 히데요시가 죽자 왜군은 퇴각을 준비한다. 이순신 장군은 명나라 장군 진린과 함께 7년간의 임진왜란을 끝낼 최후의 격전을 펼친다.',
 '개봉일': '2023년 12월 20일 (대한민국)',
 '개봉일_links': [{'text': '대한민국',
   'link': 'https://www.google.co.kr/search?sca_esv=593510951&hl=ko&gl=kr&q=%EB%8C%80%ED%95%9C%EB%AF%BC%EA%B5%AD&stick=H4sIAAAAAAAAAOPgE-LVT9c3NMzKM0oxMSkrUuLQz9U3MCtMMdYyyk620k_LzMkFE1YlGamJJUWZyYk5CkWp6Zn5eQqJ5YlFqUBOTmpicapCSmJJ6iJWntc9DW-nznm9fs-rrWt3sDLuYmfiYAAAaBmze2YAAAA&sa=X&ved=2ahUKEwiD17Kc2KmDAxUCJzQIHVRzBYcQmxMoAHoECFEQAg'}],
 '감독': '김한민',
 '감독_links': [{'text': '김한민',
   'link': 'https://www.google.co.kr/search?sca_esv=593510951&hl=ko&gl=kr&q=%EA%B9%80%ED%95%9C%EB%AF%BC&si=ALGXSlYzFQQn5id74gU-GPAR8UsllKNebHx5zj2txQsc1FfqFJgJgQJRnAVGQurej0QRiZc0Arik8YWxBF3vAm351FOItVZ-msT0tvpMZr2GX24f7uYt8NQEdMKrhRWSN

In [9]:
results['top_stories']

[{'title': "영화 '노량', 크리스마스에 200만 관객 돌파",
  'link': 'https://www.edaily.co.kr/news/read?newsId=01111926635842128&mediaCodeNo=258',
  'source': '이데일리',
  'date': '2시간 전',
  'thumbnail': 'https://serpapi.com/searches/6588fccd5119a6f342fc4d6b/images/eaa60b4c111e292489db032fc6e47bfb6249557bf146162f.jpeg'},
 {'title': "'노량' 크리스마스에 200만 돌파 -",
  'link': 'https://www.mk.co.kr/star/movies/view/2023/12/982093/',
  'source': '매일경제',
  'date': '2시간 전',
  'thumbnail': 'https://serpapi.com/searches/6588fccd5119a6f342fc4d6b/images/eaa60b4c111e2924aba457dbf47498988ccdeea91af1c731.jpeg'},
 {'title': '이순신이 총탄을 두 번 맞았다? 영화 ‘노량’과 실제 역사 비교해보니',
  'link': 'https://www.chosun.com/culture-life/relion-academia/2023/12/25/Z6PVGZD47FDCHLZS6BGE6MEHQA/',
  'source': '조선일보',
  'date': '9시간 전',
  'thumbnail': 'https://serpapi.com/searches/6588fccd5119a6f342fc4d6b/images/eaa60b4c111e292469179d09521a47d5ae434a14e5ce6dcc.jpeg'},
 {'title': '크리스마스에도 이순신 열풍! 김윤석→공명, ‘노량’ 200만 돌파에 감사 인사',
  'link': 'https://imnews.imbc.co

In [2]:
from Tools import search

temp = search.get_serpapi_search('크리스토퍼 놀란')

In [8]:
import re


exclude_patterns = [r'links', r'image', r'link', r'source']
filtered_dict = {
    key: value 
    for key, value in temp['search_results'].items() 
    if not any(re.search(pattern, key) for pattern in exclude_patterns)
}
print(filtered_dict)

{'title': '크리스토퍼 놀란', 'type': '영화 감독', 'description': '크리스토퍼 에드워드 놀란은 영국의 영화 감독, 각본가, 영화 제작자이다. 유니버시티 칼리지 런던을 졸업했다. 《미행》으로 데뷔하여 두 번째 작품 《메멘토》로 크게 성공했다. 이후 작품으로는 《인셉션》, 다크 나이트 3부작 과《인터스텔라》가 있다.', '출생': '1970년 7월 30일 (53세), 영국 런던 웨스트민스터', '형제자매': '조너선 놀런, 매튜 프란시스 놀란', '자녀': '플로라 놀런, 매그너스 놀런, 올리버 놀런, 로리 놀런', '수상': '엠파이어상 영화상, 미국 영화 연구소상 올해의 영화, 더보기', '배우자': '엠마 토마스 (1997년–)', '영화': [{'name': '오펜하이머 (2023년)', 'extensions': ['2023년'], 'link': 'https://www.google.co.kr/search?sca_esv=593529204&gl=kr&hl=ko&q=%EC%98%A4%ED%8E%9C%ED%95%98%EC%9D%B4%EB%A8%B8&stick=H4sIAAAAAAAAAAFAAL__CAMSCS9tLzAxODRkdCINL2cvMTFyZDVobTQwbioNUGVyc29uVG9Nb3ZpZaIFD-yYpO2OnO2VmOydtOuouLgFARb7Jr1AAAAA&sa=X&ved=2ahUKEwib8Nam-KmDAxUOMlkFHWGLDLAQxA16BAg6EAM', 'serpapi_link': 'https://serpapi.com/search.json?device=desktop&engine=google&gl=kr&google_domain=google.co.kr&hl=ko&location=Seoul%2C+Seoul%2C+South+Korea&q=%EC%98%A4%ED%8E%9C%ED%95%98%EC%9D%B4%EB%A8%B8&stick=H4sIAAAAAAAAAAFAAL__CAMSCS9tLzAxODRkdCINL2cvMTFyZDVobTQwbi

In [15]:
import re

def filter_dict(dictionary, exclude_patterns):
    if not isinstance(dictionary, dict):
        return dictionary

    filtered = {}
    for key, value in dictionary.items():
        if isinstance(value, dict):
            filtered[key] = filter_dict(value, exclude_patterns)
        elif isinstance(value, list):
            filtered[key] = [filter_dict(item, exclude_patterns) for item in value]
        else:
            if not any(re.search(pattern, key) for pattern in exclude_patterns):
                filtered[key] = value
    return filtered

exclude_patterns = [r'links', r'image', r'link', r'source', r'extensions']

filtered_dict = filter_dict(temp['search_results'], exclude_patterns)
filtered_dict

{'title': '크리스토퍼 놀란',
 'type': '영화 감독',
 'header_images': [{}, {}, {}, {}, {}, {}, {}],
 'description': '크리스토퍼 에드워드 놀란은 영국의 영화 감독, 각본가, 영화 제작자이다. 유니버시티 칼리지 런던을 졸업했다. 《미행》으로 데뷔하여 두 번째 작품 《메멘토》로 크게 성공했다. 이후 작품으로는 《인셉션》, 다크 나이트 3부작 과《인터스텔라》가 있다.',
 'source': {'name': '위키백과'},
 '출생': '1970년 7월 30일 (53세), 영국 런던 웨스트민스터',
 '출생_links': [{'text': '영국 런던 웨스트민스터'}],
 '형제자매': '조너선 놀런, 매튜 프란시스 놀란',
 '형제자매_links': [{'text': '조너선 놀런'}, {'text': '매튜 프란시스 놀란'}],
 '자녀': '플로라 놀런, 매그너스 놀런, 올리버 놀런, 로리 놀런',
 '자녀_links': [{'text': '플로라 놀런'},
  {'text': '매그너스 놀런'},
  {'text': '올리버 놀런'},
  {'text': '로리 놀런'}],
 '수상': '엠파이어상 영화상, 미국 영화 연구소상 올해의 영화, 더보기',
 '수상_links': [{'text': '엠파이어상 영화상'}, {'text': '미국 영화 연구소상 올해의 영화'}],
 '배우자': '엠마 토마스 (1997년–)',
 '배우자_links': [{'text': '엠마 토마스'}],
 '영화': [{'name': '오펜하이머 (2023년)', 'extensions': ['2023년']},
  {'name': '다크 나이트 (2008년)', 'extensions': ['2008년']},
  {'name': '인터스텔라 (2014년)', 'extensions': ['2014년']},
  {'name': '인셉션: 코볼사의 일 (2010년)', 'extensions': ['2010년']},
  {

# 검색 툴 통합

In [1]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish

from Tools.search import get_serpapi_search
from Tools.temperature import get_current_temperature
from Tools.mbti import get_mbti_explaination

from utils import get_openai_api_key
import pandas as pd

mbti_list = """
| mbti | 한국어 |
| --- | --- |
| ISTJ | 잇티제 |
| INTJ | 인티제 |
| ESTJ | 엣티제 |
| ENTJ | 엔티제 |
| ISFJ | 잇프제 |
| INFJ | 인프제 |
| ESFJ | 엣프제 |
| ENFJ | 엔프제 |
| ISTP | 잇팁 |
| INTP | 인팁 |
| ESTP | 엣팁 |
| ENTP | 엔팁 |
| ISFP | 잇프피 |
| INFP | 인프피 |
| ESFP | 엣프피 |
| ENFP | 앤프피 |
"""

openai.api_key = get_openai_api_key()
functions = [
        format_tool_to_openai_function(f) for f in [get_current_temperature, get_serpapi_search, get_mbti_explaination]
]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant. If user ask you about movie, you should answer sincerely. \
    If you don't have information of user, suppose that the user is in Seoul and don't ask user anything like genre.\
    If needed, you can use latest movie informations {latest_movies} \
    If needed, you can reference this {mbti_list} , match personality and call 'get_mbti_explanation'\
    If user wants MORE informations, use 'get_serpapi_search' with extracted movie, director or cast name.\
    Refuse all irrelevant questions."),
    ("user", "{input}"),
])

def route(result):
	if isinstance(result, AgentFinish): # 함수를 쓰지 않기로 결정한다면 -> content 반환
		return result.return_values['output']
	else: # 함수를 쓰기로 결정한다면 -> 함수명에 따라 어떤 함수를 쓸지 결정해주고, argument를 넣은 값 반환
		tools = {
			"get_current_temperature": get_current_temperature,
   			"get_serpapi_search": get_serpapi_search,
            "get_mbti_explaination": get_mbti_explaination
		}
		return tools[result.tool].run(result.tool_input)
  
model = ChatOpenAI(temperature=0).bind(functions=functions)
information_retriever = prompt | model | OpenAIFunctionsAgentOutputParser() | route

prompt2 = ChatPromptTemplate.from_messages([
    ("system", "You are a movie recommender. Use your knowledge to recommend movies that fit\
    and rewrite with given informations {informations}. \
    ***ALWAYS USE EXACT FIGURES AND ONLY BASED ON GIVEN INFORMATIONS**\
    You must recommned more than one movie.\
    You shoud answer using only Korean."),
])
model2 = ChatOpenAI(temperature=0.4, max_tokens=1024)
rewrite = prompt2 | model2

In [2]:
informations = information_retriever.invoke({"input":"노량 영화 알아?", "latest_movies":pd.read_csv("./data/daum_movie/movie_list.csv"), "mbti_list": mbti_list})
result = rewrite.invoke({"informations": informations})
print(result.content)

저는 '노량: 죽음의 바다'와 비슷한 영화를 추천해드릴게요. 

1. '암살' - 이 영화는 대한민국 독립운동 시기인 1933년을 배경으로 한 액션 스릴러 영화입니다. 평점은 7.9이고 예매율은 35.2%입니다.

2. '왕의 남자' - 이 영화는 조선시대 왕이 되기 위해 투쟁하는 이야기를 다룬 역사 드라마입니다. 평점은 8.1이고 예매율은 27.6%입니다.

3. '국제시장' - 이 영화는 1997년 IMF 위기 시기를 배경으로 한 가족들의 이야기를 그린 드라마입니다. 평점은 9.1이고 예매율은 45.3%입니다.

4. '명량' - 이 영화는 조선시대의 명장 이순신이 일본의 대군을 상대로 한 해전을 그린 역사 영화입니다. 평점은 8.1이고 예매율은 38.5%입니다.

5. '부산행' - 이 영화는 좀비로 변한 사람들과의 사투를 그린 액션 스릴러 영화입니다. 평점은 7.6이고 예매율은 33.9%입니다.

이 영화들은 '노량: 죽음의 바다'와 비슷한 장르나 시대를 다루고 있어 관심을 가질 수 있을 것입니다.


할루시네이션 발생

In [70]:
informations = information_retriever.invoke({"input":"오늘 날씨에 맞는 영화하나 추천해봐", "latest_movies":pd.read_csv("./data/daum_movie/movie_list.csv"), "mbti_list": mbti_list})
result = rewrite.invoke({"informations": informations})
print(result.content)

현재 온도가 2.0°C입니다. 추운 날씨에는 온기를 느낄 수 있는 영화를 추천해 드릴게요. 

1. 겨울왕국 (Frozen): 겨울 풍경과 함께 따뜻한 사랑과 용기를 느낄 수 있는 애니메이션 영화입니다. 눈과 얼음으로 가득한 이야기를 통해 추운 날씨를 잊을 수 있을 거예요.

2. 터미네이터 2: 심장을 뜨겁게 달구는 액션 영화로, 추운 날씨에도 열기를 느낄 수 있을 거예요. 스릴 넘치는 전투와 스펙터클한 액션 장면으로 온기를 전해줄 거예요.

3. 겨울밤에 만나는 음악회: 추운 겨울밤에 따뜻한 음악과 함께하는 영화입니다. 클래식 음악의 아름다움과 따뜻한 이야기로 온기를 느낄 수 있을 거예요.

이 영화들을 시청하면 추운 날씨에도 온기를 느낄 수 있을 거예요. 즐겁게 영화를 감상하세요!


In [71]:
informations = information_retriever.invoke({"input":"엔티제가 좋아할 만한 영화", "latest_movies":pd.read_csv("./data/daum_movie/movie_list.csv"), "mbti_list": mbti_list})
result = rewrite.invoke({"informations": informations})
print(result.content)

엔티제가 좋아할 만한 다른 영화로는 '매트릭스'가 추천됩니다. '매트릭스'는 가상 현실 속에서 현실과의 구분이 모호해지는 이야기를 다루며, 복잡한 플롯과 철학적인 내용이 포함되어 있습니다. 또한, 주인공이 자신의 목표를 위해 현실과 가상 현실에서 전략적으로 행동하는 모습은 엔티제의 흥미를 자극할 것입니다.

또한, '인터스텔라'도 추천해드립니다. 이 영화는 우주 여행과 시공간의 특이성을 다루며, 과학적인 요소와 철학적인 내용이 함께 포함되어 있습니다. 주인공들이 우주를 탐험하며 목표를 달성하기 위해 전략을 세우는 모습은 엔티제의 흥미를 자극할 것입니다.


In [3]:
informations = information_retriever.invoke({"input":"크리스토퍼 놀란", "latest_movies":pd.read_csv("./data/daum_movie/movie_list.csv"), "mbti_list": mbti_list})
result = rewrite.invoke({"informations": informations})
print(result.content)

크리스토퍼 놀란 감독의 작품 중 추천할 만한 영화는 다음과 같습니다:

1. 오펜하이머 (2023년) - 2023년에 개봉 예정인 크리스토퍼 놀란 감독의 신작입니다.
2. 다크 나이트 (2008년) - 크리스토퍼 놀란 감독의 대표작 중 하나로, 배트맨 시리즈의 세 번째 작품입니다.
3. 인터스텔라 (2014년) - 과학적인 요소를 담은 SF 영화로, 인류의 생존을 위한 우주 여행을 그린 작품입니다.
4. 인셉션: 코볼사의 일 (2010년) - 꿈 속에서 꿈을 꾸는 특이한 개념을 다룬 작품으로, 꿈 속에서의 현실과 환상을 구분하는 이야기입니다.

이 외에도 크리스토퍼 놀란 감독의 다른 작품들도 많이 있으니, 관심이 있다면 더 찾아보시기 바랍니다.


In [8]:
informations = information_retriever.invoke({"input":"심심한데 뭐 볼까", "latest_movies":pd.read_csv("./data/daum_movie/movie_list.csv"), "mbti_list": mbti_list})
result = rewrite.stream({"informations": informations})

for t in result:
    print(t.content, end='', flush=True)

네, 영화를 보는 것은 언제나 좋은 선택입니다! 최신 영화 정보를 알려드리겠습니다.

1. "어벤져스: 엔드게임" - 이 영화는 2019년에 개봉한 마블 슈퍼히어로 영화입니다. 전 세계에서 27억 달러 이상의 수익을 올리며 대성공을 거둔 작품이에요. 액션과 스펙터클한 시각 효과로 관객들을 매료시키는데, 특히 마블 시리즈를 좋아한다면 꼭 추천드리는 영화입니다.

2. "기생충" - 이 영화는 2019년에 개봉한 한국 영화로, 국내외에서 큰 인기를 끌었습니다. 감독 봉준호의 작품으로, 사회적인 메시지를 담은 스릴러입니다. 독특한 스토리와 연기력이 인상적이며, 여러 가지 상을 수상하며 세계적으로 인정받았습니다.

3. "조커" - 이 영화는 2019년에 개봉한 미국 영화로, 배트맨 시리즈의 악당 조커의 스토리를 다룹니다. 주인공의 광기와 성장을 그린 이 작품은 깊은 인상을 남기며, 호아킨 피닉스의 연기력이 돋보입니다. 다소 어두운 분위기의 영화이지만, 감동적인 면도 있어 많은 사람들에게 추천드립니다.

이 세 가지 영화는 각각 다른 장르와 메시지를 담고 있으며, 최신 영화 중에서도 큰 인기를 끌었습니다. 즐겁게 영화를 감상하시기 바랍니다!

저장소를 공유?

# RAG(보류)

In [1]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

embedding_function = SentenceTransformerEmbeddings(model_name="jhgan/ko-sroberta-multitask")
loader = CSVLoader('./data/daum_movie/movie_list.csv')
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
query = "가장 예매율이 높은 영화는?"
docs = db.similarity_search(query)
print(docs[0].page_content)

c:\Users\User\miniconda3\envs\movie\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


관람가: 15세이상관람가
제목: 르네상스 필름 바이 비욘세
평점: 평점10.0
예매율: 예매율0.1%
개봉일: 23년 12월 21일
줄거리: RENAISSANCE WORLD TOUR created a sanctuary for freedom, and shared joy, for more than 2.7 million fans.영화 〈RENAISSANCE: A FILM BY BEYONCE〉는 RENAISSANCE 월드 투어의 첫 공연이었던 스웨덴 스톡홀름부터 피날레 공연인 미국 미주리 주 캔자스시티까지의 여정을 모두 담고 있다. 비욘세(Beyonce)는 이번 투어가 만들어지는 모든 제작 과정에 연출을 주도하며, 그녀가 쌓아온 레거시를 대변하는 크리에이티브와 그녀가 이 투어를 하게 된 이유 등이 이 공연 실황에 모두 녹아 있다. 찬사가 끊이지 않는 비욘세(Beyonce)의 이번 〈RENAISSANCE〉 월드 투어는 270만명 이상의 팬들에게 단순한 공연이 아닌 '자유의 안식처'이 되어, 팬들과 함께 현장에서 환희와 열기를 나눴다.


In [1]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma

embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")
loader = CSVLoader('./data/netflix/netflix_titles.csv')
documents = loader.load()

db = Chroma.from_documents(documents, embedding_function)
query = "Latest TV show"
docs = db.similarity_search(query)
print(docs[0].page_content)

c:\Users\User\miniconda3\envs\movie\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


show_id: s654
type: TV Show
title: This Is Pop
director: 
cast: 
country: 
date_added: June 22, 2021
release_year: 2021
rating: TV-MA
duration: 1 Season
listed_in: Docuseries, International TV Shows
description: Uncover the real stories behind your favorite pop songs as this docuseries charts the impact of the festival scene, Auto-Tune, boy bands and more.


In [3]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# embedding_function = SentenceTransformerEmbeddings(model_name="jhgan/ko-sroberta-multitask")
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")
loader = CSVLoader('./data/netflix/netflix_titles.csv')
documents = loader.load()


In [4]:
from langchain.vectorstores import Chroma

db1 = Chroma.from_documents(docs, embedding_function, persist_directory='./index/netflix')

In [19]:
query = "Latest added TV show"
docs = db1.similarity_search(query)

In [8]:
db3 = Chroma(persist_directory="./index/netflix/", embedding_function=embedding_function)
docs = db3.similarity_search(query)
print(docs[0].page_content)

show_id: s654
type: TV Show
title: This Is Pop
director: 
cast: 
country: 
date_added: June 22, 2021
release_year: 2021
rating: TV-MA
duration: 1 Season
listed_in: Docuseries, International TV Shows
description: Uncover the real stories behind your favorite pop songs as this docuseries charts the impact of the festival scene, Auto-Tune, boy bands and more.


In [10]:
import pandas as pd
data = pd.read_csv('./data/netflix/netflix_titles.csv')

In [13]:
data['date_added'] = pd.to_datetime(data['date_added'],format='mixed')

In [21]:
data[(data['type'] == 'TV Show')&(data['release_year']==2021)].head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,2021-09-24,2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
5,s6,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",NaN,2021-09-24,2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...


In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

# embedding_function = SentenceTransformerEmbeddings(model_name="jhgan/ko-sroberta-multitask")
embedding_function = SentenceTransformerEmbeddings(model_name="BAAI/bge-small-en-v1.5")
loader = CSVLoader('./data/netflix/netflix_titles.csv')
documents = loader.load()


In [ ]:
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.agents.agent_toolkits.csv import create_csv_agent


# 대화를 기억할 메모리 추가

In [30]:
mbti_list = """
| mbti | 한국어 |
| --- | --- |
| ISTJ | 잇티제 |
| INTJ | 인티제 |
| ESTJ | 엣티제 |
| ENTJ | 엔티제 |
| ISFJ | 잇프제 |
| INFJ | 인프제 |
| ESFJ | 엣프제 |
| ENFJ | 엔프제 |
| ISTP | 잇팁 |
| INTP | 인팁 |
| ESTP | 엣팁 |
| ENTP | 엔팁 |
| ISFP | 잇프피 |
| INFP | 인프피 |
| ESFP | 엣프피 |
| ENFP | 앤프피 |
"""
prompt1_system = """
You are a intelligent and sassy retriever of informations about movies.
For successful retrieval, you can follow these steps.

- Step 1
Given User {input}, figure out the user's intentions and decide which tool to use. Try to use available tools

- Step 2
If user asked like this examples '최신영화 정보', '요새 볼만한 영화 없나',
You can give informations about latest movies or 최신 영화, use 'get_latest_movies' tool.

- Step 3
If you think user need information about personality, you can reference this {mbti_list} , match personality and use 'get_mbti_explanation' function.

- Step 4
If you think user need weather information, you can use 'get_current_temperature' function. If you don't have user information, define user is in Seoul, Korea and use the function.

- Step 5 (Optional)
If user wants MORE informations, use 'get_serpapi_search' with extracted movie, director or cast name.

Refuse all irrelevant questions.
"""
#Behave like retriever. You can't ask back user and return only informations that you retrieved.
prompt2_sytem = """
You are a kind movie recommender.
You shoud follow steps to answer.

Step 1 - From the {informations} given, you can figure out what movie the user wants.
Step 2 - Use your knowledge to movies. When you use informations given in Step 1, ***ALWAYS USE EXACT FIGURES ONLY BASED ON GIVEN informations***
Step 3 - Rewrite this into {recommend_format} format like this.

You must recommend more than one movie.
You shoud answer using only Korean.
Nice Recommendation:
"""

recommend_format = """
- <Short greetings and the reasons why you chose these movie>
- <movie name> - <plot>
- <movie name> - <plot>
- <movie name> - <plot>
- <Greetings>
"""

In [54]:
mbti_list = """
| mbti | 한국어 |
| --- | --- |
| ISTJ | 잇티제 |
| INTJ | 인티제 |
| ESTJ | 엣티제 |
| ENTJ | 엔티제 |
| ISFJ | 잇프제 |
| INFJ | 인프제 |
| ESFJ | 엣프제 |
| ENFJ | 엔프제 |
| ISTP | 잇팁 |
| INTP | 인팁 |
| ESTP | 엣팁 |
| ENTP | 엔팁 |
| ISFP | 잇프피 |
| INFP | 인프피 |
| ESFP | 엣프피 |
| ENFP | 앤프피 |
"""
prompt1_system = """
You are a intelligent and sassy retriever of informations about movies.
For successful retrieval, you can follow these steps.

- Step 1
Given User {input}, figure out the user's intentions and decide which tool to use. Try to use available tools

- Step 2
If user asked like this examples '최신영화 정보', '요새 볼만한 영화 없나',
You can give informations about latest movies or 최신 영화, use 'get_latest_movies' tool.

- Step 3
If you think user need information about personality, you can reference this {mbti_list} , match personality and use 'get_mbti_explanation' function.

- Step 4
If you think user need weather information, you can use 'get_current_temperature' function. If you don't have user information, define user is in Seoul, Korea and use the function.

- Step 5 (Optional)
If user wants MORE informations, use 'get_serpapi_search' with extracted movie, director or cast name.

Refuse all irrelevant questions.
"""
prompt2_sytem = """
You are a kind movie recommender.
You shoud follow steps to answer.

Step 1 - From the {informations} given, you can figure out what movie the user wants.
Step 2 - Use your knowledge to movies. When you use informations given in Step 1, ***ALWAYS USE EXACT FIGURES ONLY BASED ON GIVEN informations***
Step 3 - Rewrite this into {recommend_format} format like this.

You must recommend more than one movie.
You shoud answer using only Korean.
Nice Recommendation:
"""

recommend_format = """
- <Short greetings and the reasons why you chose these movie>
- <movie name> - <plot>
- <movie name> - <plot>
- <movie name> - <plot>
- <Greetings>
"""

In [55]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish
from langchain.agents import AgentExecutor
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents.format_scratchpad import format_to_openai_functions

# Tools
from Tools.search import get_serpapi_search
from Tools.temperature import get_current_temperature
from Tools.mbti import get_mbti_explaination
from Tools.latest import get_latest_movies

# utils
from utils import get_openai_api_key
import pandas as pd


openai.api_key = get_openai_api_key()
memory = ConversationBufferWindowMemory(return_messages=True, memory_key='chat_history', input_key='input', k=1)
functions = [
        format_tool_to_openai_function(f) for f in [get_current_temperature, get_serpapi_search, get_mbti_explaination, get_latest_movies]
]

prompt1 = ChatPromptTemplate.from_messages([
    ("system", prompt1_system),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

model = ChatOpenAI(temperature=0.4).bind(functions=functions)
retrieval_chain = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt1 | model | OpenAIFunctionsAgentOutputParser()

# def run_retrieval_agent(user_input):
#     intermediate_steps = []
#     while True:
#         result = retrieval_chain.invoke({
#         "input": user_input,
#         "intermediate_steps": intermediate_steps,
#         "latest_movies": pd.read_csv("./data/daum_movie/movie_list.csv"),
#         "mbti_list": mbti_list,
#     })
#         if isinstance(result, AgentFinish):
#             return result.return_values['output']
        
#         tools = {
#             "get_current_temperature": get_current_temperature,
#             "get_serpapi_search": get_serpapi_search,
#             "get_mbti_explaination": get_mbti_explaination,
#             "get_latest_movies": get_latest_movies
#         }[result.tool]
#         observation = tools.run(result.tool_input)
#         intermediate_steps.append((result, observation))
#         return intermediate_steps
    
agent_executor = AgentExecutor(agent=retrieval_chain, tools=[get_current_temperature, get_serpapi_search, get_mbti_explaination], memory=memory, verbose=False)

In [57]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.output_parsers.json import SimpleJsonOutputParser

# memory2 = ConversationBufferWindowMemory(memory_key='chat_history2', input_key='recommend_format', return_messages=False, k=1)
prompt2 = PromptTemplate(input_variables=["chat_history2", "informations", "recommend_format"],
                         template=prompt2_sytem)
model2 = ChatOpenAI(temperature=0.4, max_tokens=1024)
rewrite_chain = LLMChain(
    llm=model2,
    prompt=prompt2,
    # memory=memory2
)

In [58]:
informations = agent_executor.invoke(input={"input": "엣프피 영화 추천", "mbti_list": mbti_list})
final = rewrite_chain.invoke({"informations": informations, "recommend_format": recommend_format})
print(final['text'])

안녕하세요! 저는 영화 정보를 제공하는 역할을 하고 있습니다. 엣프피님이 영화 추천을 원하셨군요. 제가 추천해드릴 영화는 다음과 같습니다:

1. "인셉션" - 꿈 속에서 꿈을 꾸는 특수한 작전을 수행하는 팀의 이야기를 그린 영화입니다. 꿈 속에서 현실과의 경계가 흐려지는 장면들과 복잡한 스토리로 인해 시청자들을 매료시키는 작품입니다.

2. "어바웃 타임" - 시간을 되돌릴 수 있는 능력을 가진 주인공의 이야기를 그린 영화입니다. 사랑과 가족, 인생의 의미에 대해 생각해보게 되는 따뜻한 영화로, 특히 로맨스 장르를 좋아하시는 분들에게 추천드립니다.

3. "어벤져스: 엔드게임" - 마블 시네마틱 유니버스의 대미를 장식한 영화입니다. 다양한 슈퍼히어로들이 모여 최후의 전투를 펼치는 이야기로, 액션과 스펙터클한 장면들이 많이 등장하여 팬들에게 큰 호응을 얻은 작품입니다.

이상으로 추천해드릴 영화 목록입니다. 즐겁게 영화 시청하시고 좋은 시간 보내시길 바랍니다!


In [59]:
informations

{'input': '엣프피 영화 추천',
 'mbti_list': '\n| mbti | 한국어 |\n| --- | --- |\n| ISTJ | 잇티제 |\n| INTJ | 인티제 |\n| ESTJ | 엣티제 |\n| ENTJ | 엔티제 |\n| ISFJ | 잇프제 |\n| INFJ | 인프제 |\n| ESFJ | 엣프제 |\n| ENFJ | 엔프제 |\n| ISTP | 잇팁 |\n| INTP | 인팁 |\n| ESTP | 엣팁 |\n| ENTP | 엔팁 |\n| ISFP | 잇프피 |\n| INFP | 인프피 |\n| ESFP | 엣프피 |\n| ENFP | 앤프피 |\n',
 'chat_history': [],
 'output': '저는 영화 정보를 제공하는 역할을 하고 있습니다. 어떤 종류의 영화를 추천해드릴까요? 최신 영화 정보를 원하시거나, 성격 유형에 따라 추천을 받고 싶으신가요? 아니면 다른 정보가 필요하신가요?'}

In [40]:
agent_executor.invoke(input={"input": "최신 영화 정보", "mbti_list": mbti_list})

{'input': '최신 영화 정보',
 'mbti_list': '\n| mbti | 한국어 |\n| --- | --- |\n| ISTJ | 잇티제 |\n| INTJ | 인티제 |\n| ESTJ | 엣티제 |\n| ENTJ | 엔티제 |\n| ISFJ | 잇프제 |\n| INFJ | 인프제 |\n| ESFJ | 엣프제 |\n| ENFJ | 엔프제 |\n| ISTP | 잇팁 |\n| INTP | 인팁 |\n| ESTP | 엣팁 |\n| ENTP | 엔팁 |\n| ISFP | 잇프피 |\n| INFP | 인프피 |\n| ESFP | 엣프피 |\n| ENFP | 앤프피 |\n',
 'chat_history': [HumanMessage(content='엣프피 영화 추천'),
  AIMessage(content='저는 영화에 대한 정보를 제공하는 인공지능이에요. 영화 추천을 원하시나요? 최신 영화 정보를 알려드릴까요? 아니면 다른 정보가 필요하신가요?')],
 'output': '최신 영화 정보를 알려드리기 위해서는 다른 방법을 사용해야 합니다. 저는 현재 최신 영화 정보를 제공하는 기능을 갖고 있지 않아요. 다른 도움이 필요하시면 말씀해주세요!'}

In [66]:
# informations = information_retriever.invoke({"input": "ESFP가 볼만한 영화", "latest_movies": pd.read_csv("./data/daum_movie/movie_list.csv"), "mbti_list": mbti_list})
# result = rewrite.stream({"informations": informations, "recommend_format": recommend_format})

# for t in result:
#     print(t.content, end='', flush=True)

In [107]:
print(memory2.chat_memory.messages[1].content)

안녕하세요! 이병헌 영화를 찾으시는군요. 이병헌은 대한민국의 배우로 다양한 작품에서 활약하고 있습니다. 제가 추천하는 이병헌의 영화 목록은 다음과 같습니다:

1. 콘크리트 유토피아 (2023)
   - 등급: 15+
   - 상영 시간: 2시간 10분
   - 시청 옵션: 넷플릭스, TVING, YouTube, Google Play 무비/TV, 웨이브, Apple TV
   - 줄거리: 이병헌이 주연을 맡은 이 영화는 어둠 속에서 살아가는 사람들의 이야기를 그린 작품입니다.

2. 비상선언 (2021)
   - 등급: 12
   - 상영 시간: 2시간 27분
   - 줄거리: 이병헌이 주연을 맡은 이 영화는 국가 위기에 직면한 대한민국의 비상선언을 그린 작품입니다.

3. 남산의 부장들 (2020)
   - 등급: 15+
   - 상영 시간: 1시간 54분
   - 시청 옵션: 넷플릭스, 웨이브, Watcha, TVING, YouTube, Google Play 무비/TV, Apple TV
   - 줄거리: 이병헌이 주연을 맡은 이 영화는 1987년 6월의 대한민국을 배경으로 한 민주화 운동을 그린 작품입니다.

더 자세한 정보를 원하시면 해당 영화의 제목을 검색해보시기 바랍니다. 즐거운 영화 감상되세요!


In [1]:
mbti_list = """
| mbti | 한국어 |
| --- | --- |
| ISTJ | 잇티제 |
| INTJ | 인티제 |
| ESTJ | 엣티제 |
| ENTJ | 엔티제 |
| ISFJ | 잇프제 |
| INFJ | 인프제 |
| ESFJ | 엣프제 |
| ENFJ | 엔프제 |
| ISTP | 잇팁 |
| INTP | 인팁 |
| ESTP | 엣팁 |
| ENTP | 엔팁 |
| ISFP | 잇프피 |
| INFP | 인프피 |
| ESFP | 엣프피 |
| ENFP | 앤프피 |
"""
prompt1_system = """
You are a kind recommender for movies and your goal is to recommend movies. You can also retrieve informations with available tools.
Given User {input}, figure out the user's intentions and decide which tool to use. Try to use available tools
For successful retrieval, I will give some examples

- Latest movies
If user asked like this examples '최신영화 정보', '요새 볼만한 영화 없나',
You can give informations about latest movies or 최신 영화, use 'get_latest_movies' tool.

- MBTI Personality
If you think user need information about personality, you can reference this {mbti_list}. For example, 잇티제 means ISTJ, '인팁' means INTP
match personality and use 'get_mbti_explanation' function.

- Weather information
If you think user need weather information, you can use 'get_current_temperature' function. If you don't have user information, define user is in Seoul, Korea and use the function.

- More informations
If user wants MORE informations, use 'get_serpapi_search' with extracted movie, director or cast name.

When finished, go to Final Step.

- Final Step
Use what you retrieved.
When you recommend movies, you should answer based on this {recommend_format} structure.

Remember that your goal is to recommend movies!
"""

recommend_format = """
- <Short greetings>
- <movie name> - <plot>
- <movie name> - <plot>
- <movie name> - <plot>
- <Explanation about your recommendation in more than two sentences>
- <Greetings>
***ANSWER IN KOREAN!!***
"""

In [58]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish
from langchain.agents import AgentExecutor
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents.format_scratchpad import format_to_openai_functions

# Tools
from Tools.search import get_serpapi_search
from Tools.temperature import get_current_temperature
from Tools.mbti import get_mbti_explaination
from Tools.latest import get_latest_movies

# utils
from utils import get_openai_api_key
import pandas as pd


openai.api_key = get_openai_api_key()
memory = ConversationBufferWindowMemory(return_messages=True, memory_key='chat_history', input_key='input', k=5)
functions = [
        format_tool_to_openai_function(f) for f in [get_current_temperature, get_serpapi_search, get_mbti_explaination]
]

prompt1 = ChatPromptTemplate.from_messages([
    ("system", prompt1_system),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

model = ChatOpenAI(temperature=0.4).bind(functions=functions)
retrieval_chain = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt1 | model | OpenAIFunctionsAgentOutputParser()

# def run_retrieval_agent(user_input):
#     intermediate_steps = []
#     while True:
#         result = retrieval_chain.invoke({
#         "input": user_input,
#         "intermediate_steps": intermediate_steps,
#         "latest_movies": pd.read_csv("./data/daum_movie/movie_list.csv"),
#         "mbti_list": mbti_list,
#     })
#         if isinstance(result, AgentFinish):
#             return result.return_values['output']
        
#         tools = {
#             "get_current_temperature": get_current_temperature,
#             "get_serpapi_search": get_serpapi_search,
#             "get_mbti_explaination": get_mbti_explaination,
#             "get_latest_movies": get_latest_movies
#         }[result.tool]
#         observation = tools.run(result.tool_input)
#         intermediate_steps.append((result, observation))
#         return intermediate_steps
from langchain.tools import Tool
latest_movies = Tool.from_function(
    func=get_latest_movies,
    name='Latest movies',
    description='Informations about Latest movies')

agent_executor = AgentExecutor(agent=retrieval_chain, tools=[get_current_temperature, get_serpapi_search, get_mbti_explaination, latest_movies], memory=memory, verbose=True)

In [59]:
result = agent_executor.invoke({"input": "최신영화 추천", "mbti_list": mbti_list, "recommend_format":recommend_format})



> Entering new AgentExecutor chain...

Invoking: `get_latest_movies` with `{}`


get_latest_movies is not a valid tool, try one of [get_current_temperature, get_serpapi_search, get_mbti_explaination, Latest movies].
Invoking: `get_serpapi_search` with `{'query': '최신 영화'}`


{'search_results': {'새로_나온_영화': [{'name': 'REBEL MOON: 파트 1 불의 아이', 'extensions': ['15+', '2h 14m', '2023', '넷플릭스'], '시청_옵션': [{'name': '넷플릭스 구독'}]}, {'name': '헝거게임: 노래하는 새와 뱀의 발라드', 'extensions': ['15+', '2h 37m', '2023', '최저 ₩10,000'], '시청_옵션': [{'name': 'YouTube 최저 ₩10,000'}, {'name': 'Google Play 무비/TV 최저 ₩10,000'}, {'name': '웨이브 ₩11,000'}]}, {'name': '3일의 휴가', 'extensions': ['12', '1h 45m', '2023', '최저 ₩11,000'], '시청_옵션': [{'name': 'YouTube 최저 ₩11,000'}, {'name': 'Google Play 무비/TV 최저 ₩11,000'}, {'name': '웨이브 최저 ₩11,000'}]}, {'name': '엘리자벳과 나', 'extensions': ['15+', '2h 12m', '2023', '최저 ₩11,000'], '시청_옵션': [{'name': 'YouTube 최저 ₩11,000'}, {'name': 'Google Play 무비/TV 최저 ₩11,000'}, {'name': '웨이브 최저 ₩11,000'}, 

1. input prompt에 최신영화 표를 같이 넣어 최신영화 정보를 넘겨주려고 시도 -> context length 문제 발생
2. tool로 최신영화 데이터 프레임을 받아오려고 했지만 valid한 툴이 아니라고함.  
3. 메모리에 직접 최신영화 정보를 넣어주려 해도 context length 문제 여전히 발생  
-> tool로 하는 수밖에 없음

In [1]:
mbti_list = """
| mbti | 한국어 |
| --- | --- |
| ISTJ | 잇티제 |
| INTJ | 인티제 |
| ESTJ | 엣티제 |
| ENTJ | 엔티제 |
| ISFJ | 잇프제 |
| INFJ | 인프제 |
| ESFJ | 엣프제 |
| ENFJ | 엔프제 |
| ISTP | 잇팁 |
| INTP | 인팁 |
| ESTP | 엣팁 |
| ENTP | 엔팁 |
| ISFP | 잇프피 |
| INFP | 인프피 |
| ESFP | 엣프피 |
| ENFP | 앤프피 |
"""
prompt1_system = """
You are a kind recommender for movies and your goal is to recommend movies. You can also retrieve informations with available tools.
Given User {input}, figure out the user's intentions and decide which tool to use. Try to use available tools
For successful retrieval, I will give some examples

- Latest movies
If user asked like this examples '최신영화 정보', '요새 볼만한 영화 없나',
You can give informations about latest movies or 최신 영화, use 'get_latest_movies' tool.

- MBTI Personality
If you think user need information about personality, you can reference this {mbti_list}. For example, 잇티제 means ISTJ, '인팁' means INTP
match personality and use 'get_mbti_explanation' function.

- Weather information
If you think user need weather information, you can use 'get_current_temperature' function. If you don't have user information, define user is in Seoul, Korea and use the function.

- More informations
If user wants MORE informations, use 'get_serpapi_search' with extracted movie, director or cast name.

When finished, go to Final Step.

- Final Step
Use what you retrieved.
When you recommend movies, you should answer based on this {recommend_format} structure.

Remember that your goal is to recommend movies!
"""

recommend_format = """
- <Short greetings>
- <movie name> - <plot>
- <movie name> - <plot>
- <movie name> - <plot>
- <Explanation about your recommendation in more than two sentences>
- <Greetings>
***ANSWER IN KOREAN!!***
"""

In [2]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.schema.agent import AgentFinish
from langchain.agents import AgentExecutor
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferWindowMemory
from langchain.agents.format_scratchpad import format_to_openai_functions

# Tools
from Tools.search import get_serpapi_search
from Tools.temperature import get_current_temperature
from Tools.mbti import get_mbti_explaination
from Tools.latest import LatestmovieTool

# utils
from utils import get_openai_api_key
import pandas as pd


openai.api_key = get_openai_api_key()
memory = ConversationBufferWindowMemory(return_messages=True, memory_key='chat_history', input_key='input', k=5)
functions = [
        format_tool_to_openai_function(f) for f in [get_current_temperature, get_serpapi_search, get_mbti_explaination]
]

prompt1 = ChatPromptTemplate.from_messages([
    ("system", prompt1_system),
    MessagesPlaceholder(variable_name='chat_history'),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

model = ChatOpenAI(temperature=0.4).bind(functions=functions)
retrieval_chain = RunnablePassthrough.assign(
    agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt1 | model | OpenAIFunctionsAgentOutputParser()

# def run_retrieval_agent(user_input):
#     intermediate_steps = []
#     while True:
#         result = retrieval_chain.invoke({
#         "input": user_input,
#         "intermediate_steps": intermediate_steps,
#         "latest_movies": pd.read_csv("./data/daum_movie/movie_list.csv"),
#         "mbti_list": mbti_list,
#     })
#         if isinstance(result, AgentFinish):
#             return result.return_values['output']
        
#         tools = {
#             "get_current_temperature": get_current_temperature,
#             "get_serpapi_search": get_serpapi_search,
#             "get_mbti_explaination": get_mbti_explaination,
#             "get_latest_movies": get_latest_movies
#         }[result.tool]
#         observation = tools.run(result.tool_input)
#         intermediate_steps.append((result, observation))
#         return intermediate_steps

agent_executor = AgentExecutor(agent=retrieval_chain, tools=[get_current_temperature, get_serpapi_search, get_mbti_explaination, LatestmovieTool()], memory=memory, verbose=True)

In [4]:
result = agent_executor.invoke({"input": "평점이 어떤대?", "mbti_list": mbti_list, "recommend_format":recommend_format})



> Entering new AgentExecutor chain...
각 영화의 평점은 다음과 같습니다.

1. 노량: 죽음의 바다 - 평점: 8.5/10
2. 서울의 봄 - 평점: 9.2/10
3. 아쿠아맨과 로스트 킹덤 - 평점: 7.9/10

이 영화들은 대체로 높은 평점을 받았으며, 관객들로부터 좋은 평가를 받았습니다. 특히 '서울의 봄'은 높은 평점을 받아 극찬을 받고 있습니다. 어떤 영화에 대해 더 알고 싶으신가요?

> Finished chain.


In [8]:
result = agent_executor.invoke({"input": "엔프피가 볼만한 영화 추천해줘", "mbti_list": mbti_list, "recommend_format":recommend_format})



> Entering new AgentExecutor chain...
안녕하세요! 엔프피분께서 볼만한 영화를 추천해드릴게요.

1. 어바웃 타임 - 시간을 되돌릴 수 있는 능력을 가진 주인공이 사랑과 인생의 의미를 발견하는 이야기입니다. 로맨틱한 요소와 감동적인 이야기가 매력적인 영화입니다.

2. 라라랜드 - 현대의 꿈을 꾸는 사람들의 이야기를 담은 뮤지컬 영화입니다. 음악과 춤으로 표현되는 멋진 연출과 아름다운 색감이 인상적입니다.

3. 인셉션 - 꿈 속에서 꿈을 꾸는 특수한 작전을 수행하는 팀의 이야기입니다. 복잡한 스토리와 시각적인 효과가 돋보이는 영화로, 스릴과 액션을 동시에 즐길 수 있습니다.

이 영화들은 각각 다양한 장르와 흥미로운 이야기를 가지고 있습니다. 엔프피분의 취향에 맞는 영화일 것으로 추천해드립니다. 즐거운 영화 시간 보내세요!

> Finished chain.


In [9]:
result = agent_executor.invoke({"input": "오늘 날씨에 볼만한 영화 추천해줘", "mbti_list": mbti_list, "recommend_format":recommend_format})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.5665, 'longitude': 126.978}`


The current temperature is 1.9°C오늘은 날씨가 조금 춥네요. 추위를 피해 실내에서 보기 좋은 영화를 추천해드릴게요.

1. 겨울왕국 - 겨울 풍경과 따뜻한 가족 이야기가 담긴 애니메이션 영화입니다. 사랑과 용기를 다룬 이야기로, 따뜻한 감정을 전해줄 것입니다.

2. 미드나이트 인 파리 - 파리의 아름다운 풍경과 로맨틱한 이야기가 펼쳐지는 영화입니다. 파리의 밤을 배경으로 한 재미있는 시간 여행 이야기를 즐길 수 있습니다.

3. 인터스텔라 - 우주 여행을 다룬 과학적인 요소가 있는 영화입니다. 환상적인 시각 효과와 감동적인 이야기가 함께하는 SF 영화로, 시간을 가지고 여행하는 이야기가 흥미로울 것입니다.

이 영화들은 실내에서 편안하게 즐길 수 있는 영화들이에요. 온기를 느끼며 영화를 즐기시길 바랍니다. 즐거운 영화 시간 보내세요!

> Finished chain.
